In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/goodreadsbooks/books.csv


# building a book recommendation system/engine

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('/kaggle/input/goodreadsbooks/books.csv',encoding='unicode_escape',error_bad_lines=False)
df.head()

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


# create alist of columns to keep

In [5]:
columns=['title','authors','publisher']

In [6]:
def combine_features(data):
    features=[]
    for i in range(0,data.shape[0]):
        features.append(data['title'][i] + ' '+data['authors'][i]+' '+data['publisher'][i])
    return features

In [7]:
df['combined']=combine_features(df)
df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,combined
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPrÃ©,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Harry Potter and the Half-Blood Prince (Harry ...
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPrÃ©,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,Harry Potter and the Order of the Phoenix (Har...
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,Harry Potter and the Chamber of Secrets (Harry...
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPrÃ©,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,Harry Potter and the Prisoner of Azkaban (Harr...
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPrÃ©,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,Harry Potter Boxed Set Books 1-5 (Harry Potte...


# converting the text from the new column to a matrix of word counts

In [8]:
cm=CountVectorizer().fit_transform(df['combined'])

# get the cosine similarity matrix from the count matrix

In [9]:
cs=cosine_similarity(cm)
print(cs)

[[1.         0.8229512  0.73029674 ... 0.07856742 0.         0.        ]
 [0.8229512  1.         0.78881064 ... 0.14547859 0.         0.        ]
 [0.73029674 0.78881064 1.         ... 0.0860663  0.         0.        ]
 ...
 [0.07856742 0.14547859 0.0860663  ... 1.         0.2981424  0.        ]
 [0.         0.         0.         ... 0.2981424  1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [10]:
Title=df['title'][1]
Title

'Harry Potter and the Order of the Phoenix (Harry Potter  #5)'

In [11]:
book_id=df[df.title==Title]['bookID'].values[0]
book_id

2

# create a list of tuples in the form (bookID,similarity score)

In [12]:
scores=list(enumerate(cs[book_id]))
print(scores)

[(0, 0.7302967433402213), (1, 0.7888106377466154), (2, 0.9999999999999998), (3, 0.7911548052852398), (4, 0.6666666666666665), (5, 0.32274861218395134), (6, 0.778498944161523), (7, 0.15491933384829668), (8, 0.13912166872805046), (9, 0.14638501094227996), (10, 0.1369306393762915), (11, 0.11846977555181845), (12, 0.08606629658238703), (13, 0.0), (14, 0.06262242910851494), (15, 0.0), (16, 0.0), (17, 0.07161148740394328), (18, 0.0), (19, 0.0), (20, 0.1333333333333333), (21, 0.26919095102908275), (22, 0.273861278752583), (23, 0.2927700218845599), (24, 0.2656844656620285), (25, 0.2369395511036369), (26, 0.19999999999999996), (27, 0.0), (28, 0.0), (29, 0.059234887775909226), (30, 0.19364916731037082), (31, 0.1333333333333333), (32, 0.0778498944161523), (33, 0.0), (34, 0.0), (35, 0.10540925533894598), (36, 0.10540925533894598), (37, 0.11547005383792514), (38, 0.1632993161855452), (39, 0.10540925533894598), (40, 0.09128709291752767), (41, 0.10540925533894598), (42, 0.19364916731037082), (43, 0.0

# sort the similar books in descending order

In [13]:
sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
sorted_scores=sorted_scores[1:]
sorted_scores

[(4415, 0.8451542547285167),
 (10675, 0.7999999999999998),
 (3, 0.7911548052852398),
 (1, 0.7888106377466154),
 (6, 0.778498944161523),
 (1233, 0.7745966692414832),
 (0, 0.7302967433402213),
 (615, 0.7100469468046929),
 (10674, 0.7100469468046929),
 (4, 0.6666666666666665),
 (8873, 0.6515837655350015),
 (988, 0.6210590034081188),
 (10343, 0.5999999999999999),
 (10346, 0.5999999999999999),
 (4412, 0.5809475019311124),
 (4411, 0.5477225575051661),
 (10345, 0.533113989983183),
 (10344, 0.4954336943068622),
 (613, 0.4845437118523489),
 (5744, 0.45643546458763834),
 (4562, 0.408248290463863),
 (614, 0.40824829046386296),
 (4413, 0.3944053188733077),
 (5350, 0.38924947208076144),
 (4619, 0.36514837167011066),
 (5343, 0.36514837167011066),
 (5346, 0.36514837167011066),
 (3858, 0.3580574370197164),
 (8452, 0.35445877847928325),
 (8743, 0.34641016151377546),
 (8263, 0.34503277967117707),
 (4242, 0.3442651863295481),
 (5224, 0.3442651863295481),
 (7349, 0.3442651863295481),
 (6439, 0.33333333333

# create a loop to print the first 5 books from the sorted list

In [14]:
j=0
print('the 5 most recommended books to '+Title+'are:\n')
for item in scores:
    book_title=df[df.bookID==item[0]]['title'].values
    print(j+1,book_title)
    j=j+1
    if j>= 5:
        break

the 5 most recommended books to Harry Potter and the Order of the Phoenix (Harry Potter  #5)are:

1 []
2 ['Harry Potter and the Half-Blood Prince (Harry Potter  #6)']
3 ['Harry Potter and the Order of the Phoenix (Harry Potter  #5)']
4 []
5 ['Harry Potter and the Chamber of Secrets (Harry Potter  #2)']
